In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the path to your dataset containing fire and non-fire images
train_data_dir = 'train'
test_data_dir = 'test'

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    subset='training',
    class_mode='binary'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    subset='validation',
    class_mode='binary'
)

# Create a pre-trained MobileNetV2 model as the base
base_model = MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(img_width, img_height, 3)
)

# Add custom layers for classification
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Save the model for future use
model.save('fire_detection_model.h5')

# Optional: Create a test generator and evaluate the model
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


Found 1428 images belonging to 2 classes.
Found 357 images belonging to 2 classes.
9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10
45/45 [==============================] - 358s 8s/step - loss: 0.1720 - accuracy: 0.9356 - val_loss: 0.0527 - val_accuracy: 0.9804
Epoch 2/10
45/45 [==============================] - 331s 7s/step - loss: 0.0323 - accuracy: 0.9895 - val_loss: 0.0228 - val_accuracy: 0.9888
Epoch 3/10
45/45 [==============================] - 314s 7s/step - loss: 0.0177 - accuracy: 0.9930 - val_loss: 0.0189 - val_accuracy: 0.9944
Epoch 4/10
45/45 [==============================] - 315s 7s/step - loss: 0.0122 - accuracy: 0.9958 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 5/10
45/45 [==============================] - 313s 7s/step - loss: 0.0059 - accuracy: 0.9986 - val_loss: 0.0060 - val_accuracy: 1.0000
Epoch 6/10
45/45 [==============================] - 315s 7s/step - loss: 0.0048 - accuracy: 0.9986 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 7/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 515 images belonging to 2 classes.
17/17 [==============================] - 71s 4s/step - loss: 0.1311 - accuracy: 0.9709
Test Accuracy: 0.9709, Test Loss: 0.1311


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2         # Library for openCV
import threading   # Library for threading -- which allows code to run in backend
import pygame   # Library for alarm sound
import smtplib     # Library for email sending
import urllib.request
import urllib.error
from email.message import EmailMessage
import ssl

fire_cascade = cv2.CascadeClassifier('fire_detection_cascade_model.xml') # To access xml file which includes positive and negative images of fire. (Trained images)

vid = cv2.VideoCapture(0) # To start camera this command is used "0" for laptop inbuilt camera and "1" for USB attahed camera for pc
runOnce = False # created boolean

# define rez
resTrack = (640,480) #or whatever
resWrite = (1280, 720)
vid.set( cv2.CAP_PROP_FRAME_WIDTH, resWrite[0])
vid.set( cv2.CAP_PROP_FRAME_HEIGHT, resWrite[1])

def play_alarm_sound_function():
    pygame.init()  # Initialize the video system (even though we're working with audio)
    pygame.mixer.init()
    pygame.mixer.music.load('Alarm Sound.mp3')
    pygame.mixer.music.play()
    pygame.event.wait()


# def send_mail_function(): # defined function to send mail post fire detection using threading

#     email_sender = 'qwicklab321@gmail.com'
#     email_password = 'Qwicklab@123'
#     email_reciver = 'bhaveshpatil3214@gmail.com'

#     subject = 'Fire is detected'
#     body = '"Warning: Fire accident has been detected. Please take necessary actions."'


#     em = EmailMessage()

#     em['from'] = email_sender
#     em['to'] = email_reciver
#     em['subject'] = subject
#     em.set_content(body)

#     content = ssl.create_default_context()

#     with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=content) as smtp:
#         smtp.login(email_sender, email_password)
#         smtp.sendmail(email_sender, email_reciver, em.as_string())


# def internet_on():
#     try:
#         response = urllib.request.urlopen('https://www.google.co.in', timeout=1)
#         return True
#     except urllib.error.URLError as err:
#         pass
#     return False


while(True):
    Alarm_Status = False
    ret, frame = vid.read() # Value in ret is True # To read video frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # To convert frame into gray color
    fire = fire_cascade.detectMultiScale(frame, 1.2, 5) # Haar Cascade Classifier

    ## to highlight fire with square
    # Inside the loop where fire is detected
    for (x, y, w, h) in fire:
        cv2.rectangle(frame, (x - 20, y - 20), (x + w + 20, y + h + 20), (0, 0, 255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Calculate confidence based on average pixel intensity
        confidence = roi_gray.mean()

        cv2.putText(frame, f'Fire: {confidence:.2f}', (x - 20, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        print("Fire alarm initiated")
        threading.Thread(target=play_alarm_sound_function).start()  # To call alarm thread

        # if runOnce == False:
        #     print("Mail send initiated")
        #     threading.Thread(target=send_mail_function).start()  # To call alarm thread
        #     runOnce = True

        # if runOnce == True:
        #     print("Mail is already sent once")
        #     print(internet_on())
        #     runOnce = True

    cv2.imshow('Fire Detection System', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
